# BIOMASS telemetry

In [1]:
import os
from datetime import datetime, timedelta
from typing import BinaryIO, Literal, Optional, Union

import bpack
import pandas as pd
from tqdm.notebook import tqdm

from biotm import tm

%load_ext watermark
%watermark -i -v -p bpack,biotm

Python implementation: CPython
Python version       : 3.9.5
IPython version      : 7.22.0

bpack: 0.7.1
biotm: 0.3.1



In [2]:
def stream_to_dataframe(file: Union[str, os.PathLike],
                        isp_type: Literal['generic', 'sar', 'platform'] = 'generic',
                        trailing_spare_bytes: Optional[int] = None) -> pd.DataFrame:
    
    with open(file, 'rb') as fh, tqdm(total=os.stat(file).st_size / 1024 ** 2, desc=file, unit=' MB') as pbar:

        stream = tm.ISPStream(fh)
        records = []

        for n, isp in enumerate(stream.iterpackets(isp_type)):
            
            record = {
                'total_size': tm.PACKET_HEADER_SIZE + isp.packet_header.pdfl + 1,
                'timestamp': pd.Timestamp('2000-01-01')
            }
            record.update(bpack.asdict(isp.packet_header))
            record.update(bpack.asdict(isp.data_field_header))
            
            if isp_type == 'sar':
         
                sar_header = isp.source_data.sar_header()
                record['fte'] = sar_header.fte
                record.update(bpack.asdict(sar_header.obt_stat))
                record.update(synch=sar_header.synch, spct=sar_header.spct, prict=sar_header.prict)
                record.update(bpack.asdict(sar_header.data_take_information_service))
                record.update(
                    {k: v for k, v in bpack.asdict(sar_header.real_time_parameter_service).items()
                     if k not in ('sspa', 'rcm', 'sidx', 'spares')}
                )
                record['user_data_size'] = len(isp.source_data.user_data())
            
            elif isp_type == 'generic':
                
                pass
                
            elif isp_type == 'platform':
            
                plt_anc_data = bpack.asdict(isp.source_data.platform_ancillary_data(trailing_spare_bytes))
                for name in ('nav_ut', 'nav_prop_pos', 'nav_prop_vel', 'iae_dse_est_quat', 'iae_dse_est_ang_rate'):
                    param = plt_anc_data.pop(name)
                    for k, v in param.items():
                        plt_anc_data[f'{name}_{k}'] = v
                for name in ('instr_heat_ctrl_therm', 'instr_mon_therm', 'comm_quat', 'comm_ang_rate'):
                    del plt_anc_data[name]
                record.update({k: plt_anc_data[k] for k in sorted(plt_anc_data)})
                
            else:
                
                raise ValueError(f"unknown ISP type: '{isp_type}'")

            record['timestamp'] += pd.to_timedelta(isp.timestamp(), 'sec')
            pbar.update(record['total_size'] / 1024 ** 2)
            records.append(record)

    df = pd.DataFrame(records)
    df.insert(1, 'offset', df['total_size'].cumsum() - df['total_size'])

    return df


Decode SAR ISP as generic ISP:

In [3]:
df = stream_to_dataframe('data/BIO_OPER_RAW_XB_SP__2510NNNN_20250101T060251.294.isp')

data/BIO_OPER_RAW_XB_SP__2510NNNN_20250101T060251.294.isp:   0%|          | 0/308.8134765625 [00:00<?, ? MB/s]

In [4]:
df.head()

,total_size,offset,timestamp,pktvers,type,dfh,pid,pcat,gf,ssc,pdfl,pusvers,serv,subserv,dest,ct,ft
0,1792,0,2025-01-01 06:03:28.165847778,0,0,1,25,10,3,0,1785,1,211,1,0,789026608,10869
1,1792,1792,2025-01-01 06:03:28.166030884,0,0,1,25,10,3,1,1785,1,211,1,0,789026608,10881
2,1792,3584,2025-01-01 06:03:28.166213989,0,0,1,25,10,3,2,1785,1,211,1,0,789026608,10893
3,1792,5376,2025-01-01 06:03:28.166397095,0,0,1,25,10,3,3,1785,1,211,1,0,789026608,10905
4,1792,7168,2025-01-01 06:03:28.166580200,0,0,1,25,10,3,4,1785,1,211,1,0,789026608,10917


Decode SAR ISP as SAR ISP:

In [5]:
df = stream_to_dataframe('data/BIO_OPER_RAW_XB_SP__2510NNNN_20250101T060251.294.isp', isp_type='sar')

data/BIO_OPER_RAW_XB_SP__2510NNNN_20250101T060251.294.isp:   0%|          | 0/308.8134765625 [00:00<?, ? MB/s]

In [6]:
df.head()

,total_size,offset,timestamp,pktvers,type,dfh,pid,pcat,gf,ssc,...,swp,swl,bitsel,baqmode,styp,txpidx,rank,tcomp,txpl,user_data_size
0,1792,0,2025-01-01 06:03:28.294261098,0,0,1,25,10,3,0,...,3382,6720,224,1,1,0,15,False,726,1698
1,1792,1792,2025-01-01 06:03:28.294586539,0,0,1,25,10,3,1,...,3382,6720,224,1,0,0,15,False,726,1698
2,1792,3584,2025-01-01 06:03:28.294911861,0,0,1,25,10,3,2,...,3382,6720,224,1,1,0,15,False,726,1698
3,1792,5376,2025-01-01 06:03:28.295237184,0,0,1,25,10,3,3,...,3382,6720,224,1,0,0,15,False,726,1698
4,1792,7168,2025-01-01 06:03:28.295562744,0,0,1,25,10,3,4,...,3382,6720,224,1,1,0,15,False,726,1698


Decode platform ancillary ISP as generic ISP:

In [7]:
df = stream_to_dataframe('data/BIO_OPER_RAW_XB_SP__2210NNNN_20250101T060241.294.isp')

data/BIO_OPER_RAW_XB_SP__2210NNNN_20250101T060241.294.isp:   0%|          | 0/0.022762298583984375 [00:00<?, ?…

In [8]:
df.head()

,total_size,offset,timestamp,pktvers,type,dfh,pid,pcat,gf,ssc,pdfl,pusvers,serv,subserv,dest,ct,ft
0,306,0,2025-01-01 06:03:18.460418701,0,0,1,22,10,3,0,299,1,210,2,0,789026598,30174
1,306,306,2025-01-01 06:03:19.460418701,0,0,1,22,10,3,1,299,1,210,2,0,789026599,30174
2,306,612,2025-01-01 06:03:20.460418701,0,0,1,22,10,3,2,299,1,210,2,0,789026600,30174
3,306,918,2025-01-01 06:03:21.460418701,0,0,1,22,10,3,3,299,1,210,2,0,789026601,30174
4,306,1224,2025-01-01 06:03:22.460418701,0,0,1,22,10,3,4,299,1,210,2,0,789026602,30174


Decode platform ancillary ISP as platform ancillary ISP:

In [9]:
df = stream_to_dataframe('data/BIO_OPER_RAW_XB_SP__2210NNNN_20250101T060241.294.isp', isp_type='platform', trailing_spare_bytes=4)

data/BIO_OPER_RAW_XB_SP__2210NNNN_20250101T060241.294.isp:   0%|          | 0/0.022762298583984375 [00:00<?, ?…

In [10]:
df.head()

,total_size,offset,timestamp,pktvers,type,dfh,pid,pcat,gf,ssc,...,nav_prop_vel_x,nav_prop_vel_y,nav_prop_vel_z,nav_ut_day,nav_ut_dayf,nav_valid,obt_synch,orbit_number,roll_steering_idx,str_qual
0,306,0,2025-01-01 06:03:18.460418701,0,0,1,22,10,3,0,...,-1318.199718,-1531.620976,7337.802240,60676.0,0.251867,0,0,0,0,0
1,306,306,2025-01-01 06:03:19.460418701,0,0,1,22,10,3,1,...,-1326.297966,-1531.144660,7336.443180,60676.0,0.251878,0,0,0,0,0
2,306,612,2025-01-01 06:03:20.460418701,0,0,1,22,10,3,2,...,-1334.394655,-1530.665426,7335.075735,60676.0,0.251890,0,0,0,0,0
3,306,918,2025-01-01 06:03:21.460418701,0,0,1,22,10,3,3,...,-1342.489774,-1530.183275,7333.699906,60676.0,0.251902,0,0,0,0,0
4,306,1224,2025-01-01 06:03:22.460418701,0,0,1,22,10,3,4,...,-1350.583317,-1529.698207,7332.315695,60676.0,0.251913,0,0,0,0,0


Compare ISP timestamp with timestamp obtained from "NAV Universal Time in MJD":

In [11]:
MJD_OFFSET = 2400000.5 

In [12]:
df['ut_timestamp'] = pd.to_datetime(df['nav_ut_day'] + df['nav_ut_dayf'] + MJD_OFFSET, unit='D', origin='julian')

In [13]:
df[['timestamp', 'ut_timestamp']]

,timestamp,ut_timestamp
0,2025-01-01 06:03:18.460418701,2025-01-01 06:02:41.294277120
1,2025-01-01 06:03:19.460418701,2025-01-01 06:02:42.294271488
2,2025-01-01 06:03:20.460418701,2025-01-01 06:02:43.294266112
3,2025-01-01 06:03:21.460418701,2025-01-01 06:02:44.294260736
4,2025-01-01 06:03:22.460418701,2025-01-01 06:02:45.294255360
...,...,...
73,2025-01-01 06:04:31.460418701,2025-01-01 06:03:54.294243072
74,2025-01-01 06:04:32.460418701,2025-01-01 06:03:55.294277888
75,2025-01-01 06:04:33.460418701,2025-01-01 06:03:56.294272512
76,2025-01-01 06:04:34.460418701,2025-01-01 06:03:57.294267136
